In [4]:
import glob
from enrich.tei import TeiReader

In [5]:
source_dir = r"C:\Users\pandorfer\Documents\Redmine\konde\thun-project\thun-data\editions\*.xml"

In [6]:
files = [x for x in glob.iglob(source_dir, recursive=False)]

In [7]:
TRAIN_DATA = []
for x in files:
    try:
        tei_doc = TeiReader(x)
        ners = tei_doc.extract_ne_offsets('.//tei:div[@type="transcript"]', './/tei:div[@type="transcript"]//tei:rs')
        TRAIN_DATA.append(ners)
    except:
        pass

start_node: .//tei:div[@type="transcript"] couldn't be found
start_node: .//tei:div[@type="transcript"] couldn't be found


In [8]:
c = 0
for x in TRAIN_DATA:
    c = c + len(x[1]['entities'])
c

22894